In [ ]:
import pandas as pd

In [ ]:
europe_demand = pd.read_csv(snakemake.input.europedemand, parse_dates=['datetime'], index_col='datetime')

In [ ]:
# pd.read_excel('resources/14490_20250611-130454.xlsx')

In [ ]:
# data from https://www.ssb.no/statbank/table/14490/

total_demand_2024 = pd.Series({
    'NO083':5746.2,
    'NO084':8818.7,
    'NO081':8758.9,
    'NO020':6503.7,
    'NO085':4233.3,
    'NO093':3443.9,
    'NO094':6122.0,
    'NO092':7227.1,
    'NO0A1':12517.4,
    'NO0A2':22996.0,
    'NO0A3':13109.3,
    'NO060':11169.8,
    'NO071':10974.4,
    'NO072':3878.8,
    'NO073':3800.8,
})

In [ ]:
demand_share_NO = total_demand_2024/total_demand_2024.sum()

In [ ]:
distributed_demand_NO = {}
for i in demand_share_NO.index:
    distributed_demand_NO[i] = europe_demand.NO*demand_share_NO[i]
    

In [ ]:
distributed_demand_NO = pd.DataFrame(distributed_demand_NO)

In [ ]:
# regular consumption and losses
distributed_demand_NO = distributed_demand_NO - distributed_demand_NO*0.05

In [ ]:
# electric transport (23 TWh)

# data from https://www.ssb.no/statbank/table/11823/

registered_vehicles_2024 = pd.Series({
    'NO083':233032,
    'NO084':519667,
    'NO081':417860,
    'NO020':374497,
    'NO085':241815,
    'NO093':186735,
    'NO094':146553,
    'NO092':244996,
    'NO0A1':348948,
    'NO0A2':444747,
    'NO0A3':219043,
    'NO060':378676,
    'NO071':205178,
    'NO072':141752,
    'NO073':81973,
})

distributed_demand_NO = distributed_demand_NO + ((registered_vehicles_2024/registered_vehicles_2024.sum())*(23*10**6)/(365*24))

In [ ]:
# Industry (70 TWh)

# data from https://www.ssb.no/statbank/table/10314/

industry_demand_2019 = pd.Series({
    'NO083':2569.2,
    'NO084':862.0,
    'NO081':888.8,
    'NO020':1091.6,
    'NO085':711.2,
    'NO093':656.3,
    'NO094':3319.3,
    'NO092':4581.8,
    'NO0A1':6424.1,
    'NO0A2':12720.9,
    'NO0A3':8867.4,
    'NO060':3946.9,
    'NO071':6348.9,
    'NO072':1052.5,
    'NO073':2232.6,
})

distributed_demand_NO = distributed_demand_NO + (industry_demand_2019/industry_demand_2019.sum())*(70*10**6)/(365*24)

In [ ]:
# Petroleum (6 TWh)

# see NUANCE draft 2 for data source

petroleum_demand = pd.Series({
    'NO083':0.0,
    'NO084':0.0,
    'NO081':0.0,
    'NO020':0.0,
    'NO085':0.0,
    'NO093':0.0,
    'NO094':0.0,
    'NO092':0.2,
    'NO0A1':3.7,
    'NO0A2':4.9,
    'NO0A3':2.05,
    'NO060':1.6,
    'NO071':0.0,
    'NO072':0.0,
    'NO073':4.4,
})

distributed_demand_NO = distributed_demand_NO + (petroleum_demand/petroleum_demand.sum())*(6*10**6)/(365*24)

In [ ]:
# Battery production and data centres (29 TWh)

# see NUANCE draft 2 for data sources

battery_factories = pd.Series({
    'NO083':0.0,
    'NO084':0.0,
    'NO081':0.0,
    'NO020':0.0,
    'NO085':0.0,
    'NO093':0.0,
    'NO094':0.0,
    'NO092':43.0,
    'NO0A1':40.0,
    'NO0A2':0.0,
    'NO0A3':0.0,
    'NO060':40.0,
    'NO071':29.0,
    'NO072':0.0,
    'NO073':0.0,
})*65*1000

data_centres = pd.Series({
    'NO083':0.0,
    'NO084':0.0,
    'NO081':0.0,
    'NO020':150.0,
    'NO085':0.0,
    'NO093':0.0,
    'NO094':840.0,
    'NO092':0.0,
    'NO0A1':240.0,
    'NO0A2':0.0,
    'NO0A3':0.0,
    'NO060':0.0,
    'NO071':0.0,
    'NO072':0.0,
    'NO073':0.0,
})*0.8*365*24

distributed_demand_NO = distributed_demand_NO + (((battery_factories + data_centres) + (29*(10**6) - (battery_factories + data_centres).sum())/15)/(365*24))

In [ ]:
distributed_demand_NO.sum().sum()/10**6

In [ ]:
europe_demand = pd.concat([europe_demand, distributed_demand_NO], axis=1)

In [ ]:
europe_demand.to_csv(snakemake.output.europedemand)